## 유튜브 데이터 전처리

이 Jupyter Notebook에서는 유튜브 비디오 데이터의 전처리를 수행합니다.  
전처리에 사용되는 데이터는 유튜브 비디오의 다음과 같은 주요 특징을 포함합니다:
- **유튜브 제목**: 비디오의 핵심 내용을 담고 있는 텍스트
- **태그**: 비디오와 관련된 주제를 나타내는 키워드
- **설명에 포함된 해시태그**: 비디오의 주제나 관련 정보를 추가적으로 제공하는 해시태그
- **유튜브 자체 카테고리**: 유튜브에서 비디오의 주제를 나타내기 위해 제공하는 기본 분류

특히, **유튜브 제목**은 NER(명명된 개체 인식) 모델을 활용하여 명사만을 추출한 후 학습에 사용할 것입니다. 이는 불필요한 단어를 제거하고, 핵심적인 정보를 기반으로 보다 정확한 임베딩을 하기 위함입니다.


In [1]:
from gliner import GLiNER
import torch
import re
import pandas as pd
from tqdm import tqdm

c:\Users\csu52\anaconda3\envs\csu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 데이터 로드
df = pd.read_csv("../data/video_sample.csv", encoding="utf-8-sig", index_col=0)
df.head()

,video_id,channel_id,video_title,video_description,video_tags,video_duration,video_published,video_category,video_info_card,video_with_ads,video_end_screen,video_cluster,crawled_date,year,month,day
0,JCqgRM-2_GU,UCU-wl-PZYxpWeDl1OIiE_hQ,어머니와 함께 방문 한 15 세 중학생 환자분의 포경수술 영상 (포경 소매법) C...,수술 적응증 상담 : 010-9190-7575\n카카오톡 상담 : gnjurolog...,"['강남비뇨기과', '박천진', '포경', '포경수술', '귀두포피', '위생관리']",75,2024-10-06,Education,0,0,0,-1,2024-10-07 17:37:39,2024,10,6
1,R0pRp-3S5Yo,UCm69xuysxVgNuRpmXCiXlPw,YURIAN (유리안) - FUCKING MY CITY 2024 ver. (Ft. ...,#YURIAN #유리안 #FUCKING_MY_CITY_2024 #Rredrain #...,[],177,2024-10-06,Music,0,0,0,-1,2024-10-07 17:37:57,2024,10,6
2,RsVKZrt2DSo,UCVcLszfUBN9rTmc0A0uDRSQ,대리모 사건이 소환한 합법화 논쟁 / KBC뉴스,불임이나 난임 부부가 브로커에게 돈을 주고 출산을 의뢰한 '대리모 범죄'가 14년 ...,"['kbc', 'kbc광주방송', '깨비씨', 'kbc 뉴스']",158,2024-10-06,News & Politics,0,0,0,-1,2024-10-07 17:38:23,2024,10,6
3,2CpgC1K7jNI,UCxCsxXsJPDMfjcXEEV09m9w,대포죽순이요????푸바오 핸펀몰카사건? 10.6 푸바오 이야기 FUBAO PANDA...,NaN,[],400,2024-10-06,Travel & Events,0,0,0,-1,2024-10-07 17:38:24,2024,10,6
4,C-jOF1sgdOg,UCYCb_mX7P1xKrB5iaT8y0Wg,"일본에 남겨진 한국의 문화유산【아시아의 숨은 혼, 백제를 가다 Part 2】",0:00 2012년 6월 18일 다카이다야마 고분\n1:43 2012년 6월 25일...,"['TJB', 'tjb', 'tjb대전방송', '엑스포로131', '엑스포로', '...",572,2024-10-07,Entertainment,0,0,0,-1,2024-10-07 17:38:24,2024,10,7


In [3]:
# 이모티콘 삭제
def remove_emojis(text):
    """
    주어진 텍스트에서 이모티콘을 제거하는 함수.
    정규 표현식을 사용하여 다양한 범위의 이모티콘을 탐지하고 공백으로 대체.
    
    Args:
        text (str): 이모티콘을 포함할 수 있는 입력 텍스트.
    
    Returns:
        str: 이모티콘이 제거된 텍스트.
    """
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # 스마일 이모티콘
                               u"\U0001F300-\U0001F5FF"  # 기호 및 기타 심볼
                               u"\U0001F680-\U0001F6FF"  # 기타 이모티콘
                               u"\U0001F1E0-\U0001F1FF"  # 국기 이모티콘
                               "]+", flags=re.UNICODE)
    # 이모티콘을 공백으로 대체
    return emoji_pattern.sub(r' ', text)

# 정규표현식 패턴을 사용하여 특수문자 제거
def remove_special_characters(text, use_token=None, use_hashtag=False):
    """
    주어진 텍스트에서 특수 문자를 제거하고 옵션에 따라 결과를 반환하는 함수.
    
    Args:
        text (str): 특수 문자를 포함할 수 있는 입력 텍스트.
        use_token (int, optional): 특정 토큰 인덱스부터 시작하여 텍스트를 정리할지 여부를 결정하는 인덱스 값. 기본값은 None.
        use_hashtag (bool, optional): 해시태그 형식으로 변환할지 여부를 결정하는 플래그. 기본값은 False.
    
    Returns:
        str: 특수 문자가 제거된 텍스트, 토큰이 지정된 경우 해당 토큰부터 시작된 텍스트, 해시태그 형식으로 변환된 텍스트.
    """

    try:
        cleaned_list = re.sub(r'[^a-zA-Z0-9가-힣\s]', '', text).split()
    except Exception as e:
        return ""
    if use_token:
        cleaned_text = " ".join(cleaned_list[use_token:])
    else:
        cleaned_text =  " ".join(cleaned_list)
    if use_hashtag:
        cleaned_text = "#"+" #".join(cleaned_text.split())
    return cleaned_text

# 해쉬태그 추출
def hashtag_extraction(text):
    """
    주어진 텍스트에서 해시태그(#)를 추출하는 함수.
    
    Args:
        text (str): 해시태그를 포함할 수 있는 입력 텍스트.
    
    Returns:
        str: 추출된 해시태그들을 공백으로 구분한 문자열.
    """

    if isinstance(text, str):
        pattern = '#([0-9a-zA-Z가-힣]*)'
        hash_w = re.compile(pattern)
        hash_tag = ["#"+hash for hash in hash_w.findall(text)]
        return ' '.join(hash_tag)
    else:
        return ''

In [4]:
# 사전 학습된 GLiNER 모델을 로드할 경로 설정
model_path = "../models/train/checkpoint-1000"

# 지정된 경로에서 GLiNER 모델을 로드
model = GLiNER.from_pretrained(model_path)

# 사용할 장치(device) 설정: CUDA(사용 가능할 경우) 아니면 CPU 사용
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델을 설정된 장치로 이동
model.to(device)


config.json not found in C:\Users\csu52\work\dothis\dothis-ai-labs\1_youtube_video_data_preprocessing\models\train\checkpoint-1000


GLiNER(
  (model): SpanModel(
    (token_rep_layer): Encoder(
      (bert_layer): Transformer(
        (model): DebertaV2Model(
          (embeddings): DebertaV2Embeddings(
            (word_embeddings): Embedding(250105, 768, padding_idx=0)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): StableDropout()
          )
          (encoder): DebertaV2Encoder(
            (layer): ModuleList(
              (0-11): 12 x DebertaV2Layer(
                (attention): DebertaV2Attention(
                  (self): DisentangledSelfAttention(
                    (query_proj): Linear(in_features=768, out_features=768, bias=True)
                    (key_proj): Linear(in_features=768, out_features=768, bias=True)
                    (value_proj): Linear(in_features=768, out_features=768, bias=True)
                    (pos_dropout): StableDropout()
                    (dropout): StableDropout()
                  )
                  (output): Debert

In [5]:
# Labels for entity prediction
labels = ["artifacts", "person", "animal", "CIVILIZATION", "organization", \
        "phone number", "address", "passport number", "email", "credit card number", \
        "social security number", "health insurance id number", 'Business/organization', \
        "mobile phone number", "bank account number", "medication", "cpf", "driver's license number", \
        "tax identification number", "medical condition", "identity card number", "national id number", \
        "ip address", "email address", "iban", "credit card expiration date", "username", \
        "health insurance number", "student id number", "insurance number", \
        "flight number", "landline phone number", "blood type", "cvv", \
        "digital signature", "social media handle", "license plate number", "cnpj", "postal code", \
        "passport_number", "vehicle registration number", "credit card brand", \
        "fax number", "visa number", "insurance company", "identity document number", \
        "national health insurance number", "cvc", "birth certificate number", "train ticket number", \
        "passport expiration date", "social_security_number", "EVENT", "STUDY_FIELD", "LOCATION", \
        "MATERIAL", "PLANT", "TERM", "THEORY", 'Analysis Requirement']

In [6]:
# 새로운 열 'pre_text'를 데이터프레임에 추가하고 초기값을 None으로 설정
df["pre_text"] = None

# tqdm을 사용하여 데이터프레임의 각 행을 반복, 진행 상황 표시
for row in tqdm(df.iterrows(), total=len(df)):
    # 각 행에서 유튜브 제목, 태그, 설명을 가져옴
    video_title = row[1].video_title
    video_tags = row[1].video_tags
    video_description = row[1].video_description
    video_category = row[1].video_category

    # 이모티콘을 제거하고 UTF-8로 인코딩된 유튜브 제목을 처리, 길이를 384자로 제한 후 공백 제거
    video_title = remove_emojis(video_title.encode('utf-8', 'ignore').decode('utf-8'))[:384].strip()
    
    # NER 모델을 사용하여 유튜브 제목에서 명사(또는 지정된 엔터티)를 추출, 공백으로 연결
    video_title = " ".join([entity["text"] for entity in model.predict_entities(video_title, labels, threshold=1.0e-5)]).strip()
    
    # 특수 문자를 제거하여 유튜브 태그를 정리
    video_tags = remove_special_characters(video_tags).strip()
    
    # 유튜브 설명에서 해시태그를 추출
    video_hashtags = hashtag_extraction(video_description).strip()
    
    # 유튜브 제목, 태그, 해시태그를 결합하여 'pre_text' 생성
    # 유튜브 비디오 카테고리를 추가, 비디오 주제를 나타냄
    pre_text = video_title + " " + video_tags + " " + video_hashtags + " [SEP] " + video_category 
    
    # 해당 행의 'pre_text' 열에 결합된 텍스트 저장
    df.at[row[0], "pre_text"] = pre_text

# 수정된 데이터프레임을 CSV 파일로 저장, UTF-8-sig 인코딩 사용
df.to_csv("../data/video_sample_add_pre.csv", encoding="utf-8-sig")


100%|██████████| 101/101 [00:06<00:00, 15.67it/s]
